In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

# llm chain = off-the-shelf chain -> meaning general purpose chain -
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(
            variable_name="history"
        ),  # 시스템 / 사람/ AI 누가 어떤 양의 메세지를 보낼지 모르지만 MessagesPlaceholder 덕분에 Memory class로 대체
        ("human", "{question}"),
    ]
)


def load_memory(_):

    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [4]:
invoke_chain("My name is Mingi")

content='Hello Mingi! How can I assist you today?'


In [3]:
invoke_chain("what is my name?")

content='Your name is Mingi. How can I assist you today, Mingi?'
